In [1]:
import sims_pars as sims

In [2]:
bn = sims.bayes_net_from_script('''
Pcore SIER {
    b0 ~ norm(12, 1)
    b1 = 0.5 # gggg
    pf ~ beta(8, 20)
    foodstore ~ binom(100, pf)
    b0r ~ norm(0, .01)
    ageA # ~ norm(20, 3)
    ageB ~ norm(30, 2)
    ps ~ beta(5, 6)
    sexA ~ cat({'m': ps, 'f': 1-ps})
    muA = b0 + b0r + b1*ageA
    bmiA ~ norm(muA, sd)
    sdB = sd * 0.5
    muB = b0 + b0r + b1*ageB
    bmiB ~ norm(muB, sdB)
}
''')

In [11]:
bn.Exo

['ageA', 'sd']

In [13]:
sims.sample(bn, cond={'sd': 0.4, 'ageA': 30})

{'sd': 0.4,
 'ageA': 30,
 'sdB': 0.2,
 'ps': 0.5235229535485977,
 'sexA': 'm',
 'ageB': 31.089168284912965,
 'b0r': -0.001950391427949617,
 'pf': 0.26788849620409055,
 'foodstore': 27,
 'b1': 0.5,
 'b0': 14.734652371356159,
 'muB': 30.27728612238469,
 'bmiB': 30.314185806016678,
 'muA': 29.73270197992821,
 'bmiA': 29.348466275365322}

In [3]:
root = sims.NodeSet('country')
node_area = root.new_child('area', as_fixed=['b0r', 'ps'], as_floating=['foodstore'])
node_area.new_child('agA', as_fixed=['ageA', 'sexA'], as_floating=['bmiA'])
node_area.new_child('agB', as_fixed=['ageB'], as_floating=['bmiB'])

sc = sims.as_simulation_core(bn, root)

root.print()
root.print_samplers()

NodeSet country
|-Fixed ['sd', 'ageA', 'b1']
|-NodeSet area
  |-Fixed ['ps', 'b0r']
  |-Floating ['foodstore', 'pf']
  |-NodeSet agA
    |-Fixed ['sexA', 'ageA']
    |-Floating ['bmiA', 'b0', 'muA']
    |-Exo ['ps']
    |-Listening ['sd', 'b1', 'b0r']
  |-NodeSet agB
    |-Fixed ['sdB', 'ageB']
    |-Floating ['muB', 'b0', 'bmiB']
    |-Listening ['sd', 'b1', 'b0r']
NodeSet country
|-Cp: bmiA, (b0r, b0, muA)| (sd, ageA, b1)
|-Cp: foodstore, (pf)
|-Cp: muB, (ageB, b0r, b0)| (b1)
|-Cp: bmiB, (sdB, ageB, b0r, b0, muB)| (sd, b1)
|-Cp: muA, (b0r, b0)| (ageA, b1)
|-NodeSet area
  |-Cp: foodstore, (pf)
  |-Cp: bmiA, (b0, muA)| (sd, ageA, b0r, b1)
  |-Cp: muB, (ageB, b0)| (b0r, b1)
  |-Fz: pf
  |-Cp: bmiB, (sdB, ageB, b0, muB)| (sd, b0r, b1)
  |-Cp: muA, (b0)| (ageA, b0r, b1)
  |-NodeSet agA
    |-Cp: bmiA, (b0, muA)| (sd, ageA, b0r, b1)
    |-Fz: b0
    |-Cp: muA, (b0)| (ageA, b0r, b1)
  |-NodeSet agB
    |-Cp: muB, (b0)| (ageB, b0r, b1)
    |-Fz: b0
    |-Cp: bmiB, (b0, muB)| (sdB, ageB, b0r

In [29]:
bn.RVRoots

['ps', 'ageB', 'b0r', 'pf', 'b0']

In [24]:
sc

AttributeError: 'SimulationCore' object has no attribute 'Root'